In [3]:
#DANIEL ABDIERY HERNANDEZ BERAS 2022-1319 ITLA
#Codigo para instalar PYODBC libreria para trabajar Python con SQL
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [91]:
#Importar las demas librerias que usaremos en el proyecto
import pandas as pd
import pyodbc
import pyarrow as pa
import redis


In [92]:
#Realizamos un try para hacer la conexion de Python a SQL, si no nos podemos conectar que lance una 
#excepcion

try:
    connection=pyodbc.connect('DRIVER={SQL Server};SERVER=DESKTOP-DONSCGR\SQLEXPRESS;DATABASE=BikeStores;Trusted_Connection=yes;')
    print("Conexion Exitosa")
except Exception as ex:
    print(ex)

Conexion Exitosa


In [93]:
#Creamos nuestro cursor
cursor=connection.cursor()


In [94]:
#Aqui hice esta query para probar la conexion entre SQL y Python, lo que hace es pedir la version de SQL
cursor.execute("SELECT @@version;")
row=cursor.fetchone()
print(row)

('Microsoft SQL Server 2019 (RTM) - 15.0.2000.5 (X64) \n\tSep 24 2019 13:48:23 \n\tCopyright (C) 2019 Microsoft Corporation\n\tExpress Edition (64-bit) on Windows 10 Home 10.0 <X64> (Build 19045: ) (Hypervisor)\n', )


In [103]:
#Aqui con la liberia pandas realizo una query en SQL y lo convierto en dataframe
df = pd.read_sql_query("SELECT * FROM sales.customers where STATE = 'NY' ", conn)
df.head()

C:\Users\Dani\AppData\Local\Temp\ipykernel_14980\1068465625.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM sales.customers where STATE = 'NY' ", conn)


,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,1,Debra,Burks,None,debra.burks@yahoo.com,9273 Thorne Ave.,Orchard Park,NY,14127
1,4,Daryl,Spence,None,daryl.spence@aol.com,988 Pearl Lane,Uniondale,NY,11553
2,6,Lyndsey,Bean,None,lyndsey.bean@hotmail.com,769 West Road,Fairport,NY,14450
3,7,Latasha,Hays,(716) 986-3359,latasha.hays@hotmail.com,7014 Manor Station Rd.,Buffalo,NY,14215
4,8,Jacquline,Duncan,None,jacquline.duncan@yahoo.com,15 Brown St.,Jackson Heights,NY,11372


In [150]:
print(df)

      customer_id first_name last_name           phone  \
0               1      Debra     Burks            None   
1               4      Daryl    Spence            None   
2               6    Lyndsey      Bean            None   
3               7    Latasha      Hays  (716) 986-3359   
4               8  Jacquline    Duncan            None   
...           ...        ...       ...             ...   
1014         1440     Ernest   Rollins            None   
1015         1441     Jamaal  Morrison            None   
1016         1442     Cassie     Cline            None   
1017         1443     Lezlie      Lamb            None   
1018         1444     Ivette     Estes            None   

                           email                   street             city  \
0          debra.burks@yahoo.com        9273 Thorne Ave.      Orchard Park   
1           daryl.spence@aol.com          988 Pearl Lane         Uniondale   
2       lyndsey.bean@hotmail.com           769 West Road          Fai

In [105]:
file_name = 'CustomersDataFrame.xlsx'
df.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [110]:
#REDIS

r = redis.Redis(host='127.0.0.1', port=6379, db=8)

In [111]:
import pickle

In [112]:
context = pa.default_serialization_context()
r.set("ClientesNY", context.serialize(df).to_buffer().to_pybytes())

C:\Users\Dani\AppData\Local\Temp\ipykernel_14980\3252597705.py:1: FutureWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pa.default_serialization_context()


True

In [117]:
r.get("ClientesNY")

b'ahora ya no tiene la info anterior'

In [118]:
context.deserialize(r.get("ClientesNY"))

C:\Users\Dani\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3466: FutureWarning: 'pyarrow.deserialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  if await self.run_code(code, result, async_=asy):


ArrowInvalid: Expected to read 1634476133 metadata bytes, but only read 14

In [128]:
#MONGODB
#Instalamos pymongo, una libreria para trabajar con mongodb en python
!pip3 install pymongo


In [162]:
#La importamos, tambien importamos el cliente de mongo
import pymongo
from pymongo import MongoClient

In [164]:
#Revisamos la version de pymongo
pymongo.__version__

'4.6.2'

In [173]:
#Realizamos la conexion al clientee
client = pymongo.MongoClient("mongodb://localhost:27017/")


In [174]:
#Con esto convertimos los datos a diccionario para mongo
data = df.to_dict(orient='records')

In [176]:
# Hacemos esto para referirnos a que base de datos y a que coleccion nos vamos a conectar
db_name = 'clientesmongo'
collection_name = 'clientesny'

In [177]:
#Hacemos la conexion
client = MongoClient('localhost', 27017)


In [178]:
#Aqui hacemos una seleccion de que base de datos nos vamos a conectar y a que colección, si estas no existen
# mongo las creara automaticamente cuaando se inserte el primer documento en ella
db = client['clientesmongo']
collection = db['clientesny']

In [179]:

collection.insert_many(data)

InsertManyResult([ObjectId('65f0c7debede4920c5a156cd'), ObjectId('65f0c7debede4920c5a156ce'), ObjectId('65f0c7debede4920c5a156cf'), ObjectId('65f0c7debede4920c5a156d0'), ObjectId('65f0c7debede4920c5a156d1'), ObjectId('65f0c7debede4920c5a156d2'), ObjectId('65f0c7debede4920c5a156d3'), ObjectId('65f0c7debede4920c5a156d4'), ObjectId('65f0c7debede4920c5a156d5'), ObjectId('65f0c7debede4920c5a156d6'), ObjectId('65f0c7debede4920c5a156d7'), ObjectId('65f0c7debede4920c5a156d8'), ObjectId('65f0c7debede4920c5a156d9'), ObjectId('65f0c7debede4920c5a156da'), ObjectId('65f0c7debede4920c5a156db'), ObjectId('65f0c7debede4920c5a156dc'), ObjectId('65f0c7debede4920c5a156dd'), ObjectId('65f0c7debede4920c5a156de'), ObjectId('65f0c7debede4920c5a156df'), ObjectId('65f0c7debede4920c5a156e0'), ObjectId('65f0c7debede4920c5a156e1'), ObjectId('65f0c7debede4920c5a156e2'), ObjectId('65f0c7debede4920c5a156e3'), ObjectId('65f0c7debede4920c5a156e4'), ObjectId('65f0c7debede4920c5a156e5'), ObjectId('65f0c7debede4920c5a156